In [1]:
import os
from pathlib import Path
import xml.etree.ElementTree as ET
import pandas as pd
import re

In [2]:
# Check if the directory with XML files exists, if not create it
xml_directory = Path('xml_files')
if not os.path.exists(xml_directory):
    os.makedirs(xml_directory)

# Check if the output directory for Excel files exists, if not create it
output_directory = Path('excel_files')
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Find all XML files in the directory
xml_files = [f for f in xml_directory.glob('*.xml')]
print(f"Found {len(xml_files)} xml-files in the '{xml_directory}' directory.")

# Take the first XML file for processing
first_xml_file = xml_files[0]
first_xml_file

Found 35 xml-files in the 'xml_files' directory.


WindowsPath('xml_files/FatturaPa  3 000009-31.01.25 HUBER  GMBH  IT03466010232_5lE5q(E101216011).XML')

In [3]:
def extract_data_from_xml_file(xml_file, verbose=True):

    if verbose:
        print('Parsing:\t\t', xml_file.name)

    tree = ET.parse(xml_file)
    root = tree.getroot()

    def strip_tag(tag):
        return tag.split('}')[-1].lower() if tag else ''

    def parse_float(s):
        if s is None:
            return None
        s = str(s).strip()
        if s == '':
            return None
        s = s.replace(' ', '')
        s = s.replace(',', '.')
        s = re.sub(r'[^0-9.]', '', s)
        try:
            return float(s)
        except Exception:
            return None

    num_indicators = ['quantita','prezzo','prezzounitario','prezzototale','aliquota','percentuale','importo','valore','amount','price','quantity']
    lines = []
    columns = set()
    for elem in root.iter():
        if strip_tag(elem.tag) != 'dettagliolinee':
            continue
        data = {}

        # Flatten one level: child with text -> key, child with subelements -> child_subkey
        for child in elem:
            tag = strip_tag(child.tag)

            # If child has subelements, flatten them
            if any(True for _ in child):
                for sub in child:
                    sk = strip_tag(sub.tag)
                    key = f'{tag}_{sk}'
                    val = sub.text.strip() if sub.text and sub.text.strip() else None
                    data[key] = val
                    columns.add(key)
            else:
                val = child.text.strip() if child.text and child.text.strip() else None
                data[tag] = val
                columns.add(tag)

        # Convert obvious numeric fields
        for k, v in list(data.items()):
            kl = k.lower()
            if 'numero' in kl or kl == 'numerolinea':
                try:
                    data[k] = int(v) if v is not None else None
                except Exception:
                    pass
            elif any(ind in kl for ind in num_indicators):
                data[k] = parse_float(v)
        lines.append(data)

    # Ensure consistent column order: sort columns with numeric-like ones next to each other
    columns = list(columns)

    # Create DataFrame and reindex to include all discovered columns
    df = pd.DataFrame(lines)

    # Reorder columns: put numerolinea first if present
    if 'numerolinea' in df.columns:
        cols = ['numerolinea'] + [c for c in df.columns if c != 'numerolinea']
        df = df[cols]

        # Extract DatiGeneraliDocumento data (prefix keys to avoid collisions)
        raw_dati_generali = {}
        for elem in root.iter():
            if strip_tag(elem.tag) == 'datigeneralidocumento':
                for child in elem:
                    tag = strip_tag(child.tag)
                    val = child.text.strip() if child.text and child.text.strip() else None
                    raw_dati_generali[tag] = val
                break
        dati_generali = {f'dati_generali_{k}': v for k, v in raw_dati_generali.items()}

        # Extract DatiOrdineAcquisto data (prefix keys to avoid collisions)
        raw_dati_ordine = {}
        for elem in root.iter():
            if strip_tag(elem.tag) == 'datiordineacquisto':
                for child in elem:
                    tag = strip_tag(child.tag)
                    val = child.text.strip() if child.text and child.text.strip() else None
                    raw_dati_ordine[tag] = val
                break
        dati_ordine = {f'dati_ordine_{k}': v for k, v in raw_dati_ordine.items()}

        # Merge both dictionaries
        dati_generali.update(dati_ordine)

        # Add DatiGeneraliDocumento fields to each line
        for line in lines:
            line.update(dati_generali)

        # Recreate DataFrame with the updated lines
        df = pd.DataFrame(lines)

        # Reorder columns: put DatiGeneraliDocumento fields first, then numerolinea
        dati_cols = list(dati_generali.keys())
        if 'numerolinea' in df.columns:
            cols = dati_cols + ['numerolinea'] + [c for c in df.columns if c not in dati_cols + ['numerolinea']]
        else:
            cols = dati_cols + [c for c in df.columns if c not in dati_cols]
        df = df[cols]

        # Drop specific unwanted columns if they exist
        for col_to_drop in ['tipodocumento', 'divisa']:
            if col_to_drop in df.columns:
                df = df.drop(columns=[col_to_drop])

        # Convert date fields to datetime if they exist
        date_fields = ['datadocumento', 'datascadenza', 'data']
        for date_field in date_fields:
            if date_field in df.columns:
                df[date_field] = pd.to_datetime(df[date_field], errors='coerce')

    if verbose:
        print('Extracted lines:\t', len(df))
        print('Discovered columns:', list(df.columns))
        print()

    return df

In [4]:
_ = extract_data_from_xml_file(first_xml_file)

Parsing:		 FatturaPa  3 000009-31.01.25 HUBER  GMBH  IT03466010232_5lE5q(E101216011).XML
Extracted lines:	 16
Discovered columns: ['dati_generali_tipodocumento', 'dati_generali_divisa', 'dati_generali_data', 'dati_generali_numero', 'dati_generali_importototaledocumento', 'dati_generali_causale', 'dati_ordine_iddocumento', 'dati_ordine_data', 'dati_ordine_codicecig', 'numerolinea', 'descrizione', 'prezzounitario', 'prezzototale', 'aliquotaiva', 'codicearticolo_codicetipo', 'codicearticolo_codicevalore', 'quantita', 'unitamisura', 'scontomaggiorazione_tipo', 'scontomaggiorazione_percentuale']



In [5]:
# Initialize an empty DataFrame to store merged data
merged_df = pd.DataFrame()

# Loop through all XML files and extract data
for xml_file in xml_files:
    df = extract_data_from_xml_file(xml_file, verbose=False)
    if df is not None and not df.empty:
        merged_df = pd.concat([merged_df, df], ignore_index=True)

# Save merged DataFrame to Excel
output_file = output_directory / 'merged_invoices.xlsx'
merged_df.to_excel(output_file, index=False)